In [1]:
from pathlib import Path
import sys  
import os
import pandas as pd 
from datetime import datetime
import scipy.sparse as sps
from numpy import linalg as LA

import numpy as np
from operator import itemgetter
import optuna as op

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [15]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from libs.Evaluation.Evaluator import EvaluatorHoldout

In [3]:
%load_ext autoreload
%autoreload 2

In [10]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [11]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [17]:
slim_model= SLIMElasticNetRecommender

In [18]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 171 ( 0.5%) Users that have less than 1 test interactions


In [33]:
def objective(trial):
    topK = trial.suggest_int("topK", 290, 310)
    l1 = trial.suggest_float("l1_ratio", 0.05, 0.25)
    alpha = trial.suggest_float("alpha", 0.0001, 0.001)

    
    recommender = slim_model(URM_train)
    recommender.fit(**trial.params)
    
    result_data, _ = evaluator_validation.evaluateRecommender(recommender)
    
    return result_data["MAP"][10]

In [34]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame()
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        #self.results_df = pd.concat([self.results_df, pd.DataFrame(hyperparam_dict).transpose()], ignore_index=True)
        self.results_df = self.results_df._append(hyperparam_dict, ignore_index=True)

In [ ]:
study = op.create_study(direction="maximize")
save_results= SaveResults()
study.optimize(objective, callbacks=[save_results], n_trials=100)

[I 2024-12-07 23:48:42,541] A new study created in memory with name: no-name-c50526c5-e41b-40c2-9e8a-53b0b3d63245


SLIMElasticNetRecommender: Processed 8662 (22.7%) in 5.00 min. Items per second: 28.87
SLIMElasticNetRecommender: Processed 17444 (45.8%) in 10.00 min. Items per second: 29.07
SLIMElasticNetRecommender: Processed 26541 (69.6%) in 15.00 min. Items per second: 29.49
SLIMElasticNetRecommender: Processed 35702 (93.7%) in 20.00 min. Items per second: 29.75
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 21.34 min. Items per second: 29.77
EvaluatorHoldout: Processed 35565 (100.0%) in 16.43 sec. Users per second: 2164


[I 2024-12-08 00:10:19,442] Trial 0 finished with value: 0.06051702784304837 and parameters: {'topK': 302, 'l1_ratio': 0.05059812759555809, 'alpha': 0.0003401723065596712}. Best is trial 0 with value: 0.06051702784304837.


SLIMElasticNetRecommender: Processed 9408 (24.7%) in 5.00 min. Items per second: 31.36
SLIMElasticNetRecommender: Processed 18882 (49.5%) in 10.00 min. Items per second: 31.47
SLIMElasticNetRecommender: Processed 28932 (75.9%) in 15.00 min. Items per second: 32.14
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 19.55 min. Items per second: 32.49
EvaluatorHoldout: Processed 35565 (100.0%) in 12.02 sec. Users per second: 2960


[I 2024-12-08 00:30:04,783] Trial 1 finished with value: 0.05945309727996162 and parameters: {'topK': 304, 'l1_ratio': 0.12776693007353476, 'alpha': 0.0004927353749257695}. Best is trial 0 with value: 0.06051702784304837.


...

SLIMElasticNetRecommender: Processed 8154 (21.4%) in 5.00 min. Items per second: 27.18
SLIMElasticNetRecommender: Processed 16702 (43.8%) in 10.00 min. Items per second: 27.83
SLIMElasticNetRecommender: Processed 25342 (66.5%) in 15.00 min. Items per second: 28.16
SLIMElasticNetRecommender: Processed 34042 (89.3%) in 20.00 min. Items per second: 28.37
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 22.40 min. Items per second: 28.37
EvaluatorHoldout: Processed 35565 (100.0%) in 14.59 sec. Users per second: 2437


[I 2024-12-08 15:11:56,219] Trial 45 finished with value: 0.06070154021587815 and parameters: {'topK': 306, 'l1_ratio': 0.09637882791980767, 'alpha': 0.0003156761386783485}. Best is trial 7 with value: 0.060795630178589.


SLIMElasticNetRecommender: Processed 8781 (23.0%) in 5.00 min. Items per second: 29.26
SLIMElasticNetRecommender: Processed 17736 (46.5%) in 10.00 min. Items per second: 29.56
SLIMElasticNetRecommender: Processed 26903 (70.6%) in 15.00 min. Items per second: 29.89
SLIMElasticNetRecommender: Processed 36011 (94.5%) in 20.00 min. Items per second: 30.01
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 21.20 min. Items per second: 29.97
EvaluatorHoldout: Processed 35565 (100.0%) in 12.85 sec. Users per second: 2768


[I 2024-12-08 15:33:21,201] Trial 46 finished with value: 0.0606725267172315 and parameters: {'topK': 303, 'l1_ratio': 0.23594557170877611, 'alpha': 0.000207507926672447}. Best is trial 7 with value: 0.060795630178589.


SLIMElasticNetRecommender: Processed 8261 (21.7%) in 5.00 min. Items per second: 27.53
SLIMElasticNetRecommender: Processed 16727 (43.9%) in 10.00 min. Items per second: 27.88
SLIMElasticNetRecommender: Processed 25017 (65.6%) in 15.00 min. Items per second: 27.79


,topK,l1_ratio,alpha,result
0,720.0,0.343464,0.768518,0.000265
1,145.0,0.930758,0.587530,0.000265
2,386.0,0.106288,0.872912,0.000265
3,26.0,0.543890,0.859017,0.000265


In [3]:
old_results = pd.read_csv("result_experiments/SLIM/results.csv")

In [5]:
old_results.sort_values(by=['MAP'], ascending=False)

,topK,l1_ratio,alpha,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
19,300,0.11800,0.00026,2024-11-30 07:54:47,10,0.111556,0.166356,0.141104,0.060479,0.087965,...,0.994991,0.569622,0.994991,0.240193,13.452813,0.999874,0.395067,0.911396,1.359627,0.243844
63,330,0.11100,0.00026,2024-12-01 06:51:24,10,0.111536,0.166336,0.141116,0.060466,0.087930,...,0.994991,0.569398,0.994991,0.240864,13.455295,0.999873,0.396171,0.911564,1.363084,0.243832
55,320,0.11625,0.00026,2024-12-01 04:02:42,10,0.111522,0.166355,0.141133,0.060464,0.087947,...,0.994991,0.569510,0.994991,0.240054,13.450461,0.999873,0.394839,0.911236,1.364541,0.243797
39,310,0.11800,0.00026,2024-11-30 22:22:20,10,0.111508,0.166290,0.141070,0.060463,0.087942,...,0.994991,0.569538,0.994991,0.239982,13.450652,0.999873,0.394719,0.911249,1.362562,0.243814
43,320,0.11100,0.00026,2024-11-30 23:49:08,10,0.111528,0.166322,0.141103,0.060460,0.087929,...,0.994991,0.569398,0.994991,0.241075,13.457495,0.999874,0.396518,0.911713,1.359875,0.243864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,350,0.11100,0.00020,2024-12-02 03:04:27,10,0.111157,0.165720,0.140639,0.060305,0.087739,...,0.994991,0.568278,0.994991,0.253919,13.534857,0.999886,0.417643,0.916954,1.326083,0.244530
20,310,0.11100,0.00020,2024-11-30 08:15:55,10,0.111084,0.165681,0.140623,0.060299,0.087732,...,0.994991,0.568027,0.994991,0.255103,13.545145,0.999889,0.419591,0.917651,1.312987,0.244680
60,330,0.11100,0.00020,2024-12-01 05:48:24,10,0.111120,0.165680,0.140613,0.060298,0.087725,...,0.994991,0.568167,0.994991,0.254458,13.539355,0.999887,0.418530,0.917259,1.320494,0.244595
40,320,0.11100,0.00020,2024-11-30 22:44:29,10,0.111103,0.165689,0.140637,0.060292,0.087724,...,0.994991,0.568223,0.994991,0.254819,13.542520,0.999888,0.419124,0.917473,1.316684,0.244639


In [7]:
old_results.sort_values(by=['RECALL'], ascending=False)

,topK,l1_ratio,alpha,time,cutoff,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
75,330,0.11625,0.00026,2024-12-01 11:03:56,10,0.111503,0.166348,0.141143,0.060451,0.087932,...,0.994991,0.569510,0.994991,0.239853,13.448351,0.999872,0.394507,0.911093,1.367301,0.243768
55,320,0.11625,0.00026,2024-12-01 04:02:42,10,0.111522,0.166355,0.141133,0.060464,0.087947,...,0.994991,0.569510,0.994991,0.240054,13.450461,0.999873,0.394839,0.911236,1.364541,0.243797
103,350,0.11100,0.00026,2024-12-02 04:07:47,10,0.111497,0.166318,0.141127,0.060441,0.087900,...,0.994991,0.569314,0.994991,0.240487,13.451387,0.999872,0.395552,0.911299,1.368347,0.243777
63,330,0.11100,0.00026,2024-12-01 06:51:24,10,0.111536,0.166336,0.141116,0.060466,0.087930,...,0.994991,0.569398,0.994991,0.240864,13.455295,0.999873,0.396171,0.911564,1.363084,0.243832
79,330,0.11800,0.00026,2024-12-01 13:17:13,10,0.111480,0.166309,0.141116,0.060447,0.087918,...,0.994991,0.569482,0.994991,0.239611,13.446661,0.999872,0.394110,0.910979,1.368187,0.243756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,320,0.11100,0.00020,2024-11-30 22:44:29,10,0.111103,0.165689,0.140637,0.060292,0.087724,...,0.994991,0.568223,0.994991,0.254819,13.542520,0.999888,0.419124,0.917473,1.316684,0.244639
80,340,0.11100,0.00020,2024-12-01 13:39:54,10,0.111162,0.165720,0.140635,0.060310,0.087736,...,0.994991,0.568306,0.994991,0.254164,13.537005,0.999887,0.418047,0.917100,1.323202,0.244562
140,370,0.11100,0.00020,2024-12-03 10:19:45,10,0.111148,0.165709,0.140625,0.060311,0.087738,...,0.994991,0.568278,0.994991,0.253398,13.529806,0.999884,0.416786,0.916612,1.333071,0.244459
20,310,0.11100,0.00020,2024-11-30 08:15:55,10,0.111084,0.165681,0.140623,0.060299,0.087732,...,0.994991,0.568027,0.994991,0.255103,13.545145,0.999889,0.419591,0.917651,1.312987,0.244680
